# Demo of PSyKI

This is a brief demonstration of PSyKI.
All main symbolic knowledge injection workflow steps will be addressed.


Some utility imports.

In [2]:
from psyki.logic.datalog.grammar.adapters.antlr4 import get_formula_from_string
from test.resources.rules.poker import FEATURE_MAPPING as POKER_FEATURE_MAPPING
from test.resources.rules.poker import CLASS_MAPPING as POKER_CLASS_MAPPING
from test.utils import get_processed_dataset
from test.resources.rules import get_rules

Loading poker dataset and separation into train and test set.

In [3]:
feature_mapping = POKER_FEATURE_MAPPING
class_mapping = POKER_CLASS_MAPPING
train_x, train_y, test_x, test_y = get_processed_dataset('poker')
train_x

array([[ 1., 10.,  1., ..., 12.,  1.,  1.],
       [ 2., 11.,  2., ..., 12.,  2.,  1.],
       [ 3., 12.,  3., ..., 10.,  3.,  1.],
       ...,
       [ 2.,  1.,  2., ...,  1.,  4., 13.],
       [ 2., 12.,  4., ..., 12.,  4.,  9.],
       [ 1.,  7.,  3., ...,  8.,  3.,  7.]])

Import Datalog poker rules and parsing into Formula data structure.

In [4]:
poker_rules = get_rules('poker')
formulae = [get_formula_from_string(rule) for rule in poker_rules]
str(formulae[0])

'pair(S1,R1,S2,R2,S3,R3,S4,R4,S5,R5)←((R1)=(R2))'

## Injection via structuring using NetworkComposer

In [5]:
from tensorflow.keras import Input
from psyki.ski.injectors import NetworkComposer
from tensorflow.keras.models import Model
from test.utils import get_mlp
from tensorflow.python.framework.random_seed import set_random_seed

set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=64, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
injector = NetworkComposer(model, feature_mapping, layer=len(model.layers)-2)
predictor = injector.inject(formulae)
predictor.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
predictor.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 10)]         0                                            
__________________________________________________________________________________________________
lambda_416 (Lambda)             (None, 1)            0           Input[0][0]                      
__________________________________________________________________________________________________
lambda_417 (Lambda)             (None, 1)            0           Input[0][0]                      
__________________________________________________________________________________________________
lambda_418 (Lambda)             (None, 1)            0           Input[0][0]                      
____________________________________________________________________________________________

### Training

In [6]:
predictor.fit(train_x, train_y, verbose=1, batch_size=32, epochs=20)

Epoch 1/20


2022-06-27 15:34:03.964476: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-27 15:34:03.967937: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


782/782 [==============================] - 8s 3ms/step - loss: 0.7106 - accuracy: 0.8305
Epoch 2/20
782/782 [==============================] - 2s 3ms/step - loss: 0.3499 - accuracy: 0.9297
Epoch 3/20
782/782 [==============================] - 2s 3ms/step - loss: 0.2089 - accuracy: 0.9543
Epoch 4/20
782/782 [==============================] - 2s 3ms/step - loss: 0.1353 - accuracy: 0.9811
Epoch 5/20
782/782 [==============================] - 2s 3ms/step - loss: 0.0913 - accuracy: 0.9921
Epoch 6/20
782/782 [==============================] - 2s 3ms/step - loss: 0.0636 - accuracy: 0.9953
Epoch 7/20
782/782 [==============================] - 2s 3ms/step - loss: 0.0447 - accuracy: 0.9974
Epoch 8/20
782/782 [==============================] - 2s 3ms/step - loss: 0.0320 - accuracy: 0.9988
Epoch 9/20
782/782 [==============================] - 3s 3ms/step - loss: 0.0232 - accuracy: 0.9994
Epoch 10/20
782/782 [==============================] - 2s 3ms/step - loss: 0.0171 - accuracy: 0.9996
Epoch 11/2

### Evaluation
Loss and accuracy

In [7]:
loss, accuracy = predictor.evaluate(test_x, test_y)
print('Loss: ' + str(loss))
print('Accuracy: ' + str(accuracy))

31250/31250 [==============================] - 79s 2ms/step - loss: 0.0022 - accuracy: 0.9997
Loss: 0.0021584308706223965
Accuracy: 0.9997450113296509


In [8]:
from test.utils import get_class_accuracy

accuracies, weights = get_class_accuracy(predictor, test_x, test_y)
for label, index in sorted(class_mapping.items(), key=lambda i: i[1]):
    print('Accuracy of class ' + label + ': ' + str(accuracies[index]))

Accuracy of class nothing: 1.0
Accuracy of class pair: 1.0
Accuracy of class two: 1.0
Accuracy of class three: 1.0
Accuracy of class straight: 0.9997425997425997
Accuracy of class flush: 1.0
Accuracy of class full: 0.9676966292134831
Accuracy of class four: 0.14782608695652175
Accuracy of class straight_flush: 0.16666666666666666
Accuracy of class royal_flush: 0.3333333333333333


Macro F1-measure

In [9]:
from test.utils import get_f1

f1 = get_f1(predictor, test_x, test_y)
print('macro F1-measure: ' + str(f1))

macro F1-measure: 0.8021688605291398


## Injection via constraining using LambdaLayer

In [11]:
from psyki.ski.injectors import LambdaLayer

str(formulae[0])
set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=64, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
injector = LambdaLayer(model, class_mapping, feature_mapping, 1.5)
predictor = injector.inject(formulae)
predictor.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
predictor.summary()

Model: "constrained_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense_619 (Dense)               (None, 64)           704         Input[0][0]                      
__________________________________________________________________________________________________
dense_620 (Dense)               (None, 64)           4160        dense_619[0][0]                  
__________________________________________________________________________________________________
dense_621 (Dense)               (None, 10)           650         dense_620[0][0]                  
__________________________________________________________________________________

### Training

In [12]:
predictor.fit(train_x, train_y, verbose=1, batch_size=32, epochs=100)

Epoch 1/100
782/782 [==============================] - 13s 5ms/step - loss: 1.4766 - accuracy: 0.2231
Epoch 2/100
782/782 [==============================] - 4s 5ms/step - loss: 1.3911 - accuracy: 0.2495
Epoch 3/100
782/782 [==============================] - 4s 5ms/step - loss: 1.3645 - accuracy: 0.2896
Epoch 4/100
782/782 [==============================] - 4s 6ms/step - loss: 1.3482 - accuracy: 0.3269
Epoch 5/100
782/782 [==============================] - 4s 6ms/step - loss: 1.3348 - accuracy: 0.3396
Epoch 6/100
782/782 [==============================] - 4s 5ms/step - loss: 1.3248 - accuracy: 0.3522
Epoch 7/100
782/782 [==============================] - 4s 5ms/step - loss: 1.3142 - accuracy: 0.3579
Epoch 8/100
782/782 [==============================] - 4s 5ms/step - loss: 1.3049 - accuracy: 0.3675
Epoch 9/100
782/782 [==============================] - 4s 5ms/step - loss: 1.2937 - accuracy: 0.3702
Epoch 10/100
782/782 [==============================] - 4s 6ms/step - loss: 1.2822 - accur

782/782 [==============================] - 4s 6ms/step - loss: 0.0988 - accuracy: 0.9737
Epoch 82/100
782/782 [==============================] - 4s 6ms/step - loss: 0.1004 - accuracy: 0.9726
Epoch 83/100
782/782 [==============================] - 4s 6ms/step - loss: 0.0925 - accuracy: 0.9764
Epoch 84/100
782/782 [==============================] - 4s 6ms/step - loss: 0.0966 - accuracy: 0.9736
Epoch 85/100
782/782 [==============================] - 5s 6ms/step - loss: 0.0960 - accuracy: 0.9734
Epoch 86/100
782/782 [==============================] - 5s 6ms/step - loss: 0.0887 - accuracy: 0.9774
Epoch 87/100
782/782 [==============================] - 5s 6ms/step - loss: 0.0909 - accuracy: 0.9750
Epoch 88/100
782/782 [==============================] - 5s 6ms/step - loss: 0.0879 - accuracy: 0.9765
Epoch 89/100
782/782 [==============================] - 4s 6ms/step - loss: 0.0853 - accuracy: 0.9772
Epoch 90/100
782/782 [==============================] - 4s 6ms/step - loss: 0.0822 - accuracy: 

### Remove constraints and evaluation
Loss and Accuracy

In [13]:
#predictor = injector.remove()
predictor.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
loss, accuracy = predictor.evaluate(test_x, test_y)
print('Loss: ' + str(loss))
print('Accuracy: ' + str(accuracy))

31250/31250 [==============================] - 12s 346us/step - loss: 0.0795 - accuracy: 0.9806
Loss: 0.07949545234441757
Accuracy: 0.9805639982223511


In [14]:
accuracies, weights = get_class_accuracy(predictor, test_x, test_y)
for label, index in sorted(class_mapping.items(), key=lambda i: i[1]):
    print('Accuracy of class ' + label + ': ' + str(accuracies[index]))

Accuracy of class nothing: 0.997743456322612
Accuracy of class pair: 0.9956520504239074
Accuracy of class two: 0.8621435471000798
Accuracy of class three: 0.8730647223142843
Accuracy of class straight: 0.05019305019305019
Accuracy of class flush: 0.000501002004008016
Accuracy of class full: 0.09269662921348315
Accuracy of class four: 0.0
Accuracy of class straight_flush: 0.0
Accuracy of class royal_flush: 0.0


Macro F1-measure

In [15]:
f1 = get_f1(predictor, test_x, test_y)
print('macro F1-measure: ' + str(f1))

macro F1-measure: 0.39584003713888716


## The same network without knowledge injection

In [16]:
set_random_seed(0)
input_features = Input((10,), name='Input')
network = get_mlp(input_layer=input_features, output=10, layers=3, neurons=64, activation_function='relu', last_activation_function='softmax')
model = Model(input_features, network)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 10)]              0         
_________________________________________________________________
dense_622 (Dense)            (None, 64)                704       
_________________________________________________________________
dense_623 (Dense)            (None, 64)                4160      
_________________________________________________________________
dense_624 (Dense)            (None, 10)                650       
Total params: 5,514
Trainable params: 5,514
Non-trainable params: 0
_________________________________________________________________


### Training

In [17]:
model.fit(train_x, train_y, verbose=1, batch_size=32, epochs=100)

Epoch 1/100
782/782 [==============================] - 0s 362us/step - loss: 1.0413 - accuracy: 0.4912
Epoch 2/100
782/782 [==============================] - 0s 355us/step - loss: 0.9710 - accuracy: 0.5263
Epoch 3/100
782/782 [==============================] - 0s 357us/step - loss: 0.9570 - accuracy: 0.5398
Epoch 4/100
782/782 [==============================] - 0s 355us/step - loss: 0.9496 - accuracy: 0.5455
Epoch 5/100
782/782 [==============================] - 0s 355us/step - loss: 0.9406 - accuracy: 0.5541
Epoch 6/100
782/782 [==============================] - 0s 356us/step - loss: 0.9340 - accuracy: 0.5612
Epoch 7/100
782/782 [==============================] - 0s 356us/step - loss: 0.9261 - accuracy: 0.5595
Epoch 8/100
782/782 [==============================] - 0s 359us/step - loss: 0.9191 - accuracy: 0.5701
Epoch 9/100
782/782 [==============================] - 0s 358us/step - loss: 0.9123 - accuracy: 0.5746
Epoch 10/100
782/782 [==============================] - 0s 358us/step - l

782/782 [==============================] - 0s 357us/step - loss: 0.0740 - accuracy: 0.9832
Epoch 80/100
782/782 [==============================] - 0s 357us/step - loss: 0.0793 - accuracy: 0.9804
Epoch 81/100
782/782 [==============================] - 0s 356us/step - loss: 0.0682 - accuracy: 0.9846
Epoch 82/100
782/782 [==============================] - 0s 356us/step - loss: 0.0747 - accuracy: 0.9814
Epoch 83/100
782/782 [==============================] - 0s 358us/step - loss: 0.0725 - accuracy: 0.9832
Epoch 84/100
782/782 [==============================] - 0s 358us/step - loss: 0.0671 - accuracy: 0.9839
Epoch 85/100
782/782 [==============================] - 0s 457us/step - loss: 0.0701 - accuracy: 0.9834
Epoch 86/100
782/782 [==============================] - 0s 358us/step - loss: 0.0591 - accuracy: 0.9873
Epoch 87/100
782/782 [==============================] - 0s 359us/step - loss: 0.0649 - accuracy: 0.9848
Epoch 88/100
782/782 [==============================] - 0s 359us/step - loss:

### Evaluation
Loss and Accuracy

In [18]:
loss, accuracy = model.evaluate(test_x, test_y)
print('Loss: ' + str(loss))
print('Accuracy: ' + str(accuracy))

31250/31250 [==============================] - 6s 197us/step - loss: 0.1261 - accuracy: 0.9664
Loss: 0.1261490285396576
Accuracy: 0.9664030075073242


In [19]:
accuracies, weights = get_class_accuracy(model, test_x, test_y)
for label, index in sorted(class_mapping.items(), key=lambda i: i[1]):
    print('Accuracy of class ' + label + ': ' +str(accuracies[index]))

Accuracy of class nothing: 0.9944434357723025
Accuracy of class pair: 0.966158419684827
Accuracy of class two: 0.8478224350090294
Accuracy of class three: 0.871028833862033
Accuracy of class straight: 0.15933075933075933
Accuracy of class flush: 0.011523046092184368
Accuracy of class full: 0.2478932584269663
Accuracy of class four: 0.05217391304347826
Accuracy of class straight_flush: 0.0
Accuracy of class royal_flush: 0.0


Macro F1-measure

In [20]:
f1 = get_f1(model, test_x, test_y)
print('macro F1-measure: ' + str(f1))

macro F1-measure: 0.4355657347081636
